In [1]:
### lib for arrays with cols and rows
import pandas as pd

columns = ["length", "helpful", "emotional", "Sentiment"]

data = [
    ["long", "yes", "yes", "positive"],
    ["short", "no", "no", "negative"],
    ["long", "yes", "yes", "positive"],
    ["long", "yes", "no", "negative"],
    ["long", "no", "yes", "positive"],
    ["short", "yes", "yes", "positive"],
    ["long", "no", "no", "negative"],
    ["long", "no", "no", "positive"],
    ["long", "yes", "yes", "positive"],
    ["long", "no", "no", "positive"], 
    ["long", "no", "no", "positive"],
]

other_data_1 = [
    ["long", "yes", "yes", "positive"],
    ["short", "no", "no", "negative"],
    ["long", "yes", "yes", "positive"],
    ["long", "yes", "no", "positive"],
]

other_data_2 = [
    ["short", "yes", "yes", "positive"],
    ["short", "no", "no", "negative"],
    ["long", "no", "no", "positive"],
    ["long", "yes", "yes", "positive"],
]

other_data_3 = [
    ["short", "no", "no", "negative"], 
    ["long", "no", "no", "positive"],
    ["short", "yes", "no", "negative"],
    ["long", "no", "yes", "positive"],
]

rows = ["Review 1","Review 2", "Review 3", "Review 4", "Review 5", "Review 6",
       "Review 7", "Review 8", "Review 9", "Review 10", "Review 11"]


df = pd.DataFrame(data = data, columns = columns, index = rows)
df2 = pd.DataFrame(other_data_1, columns = columns, index = rows[0:4])
df3 = pd.DataFrame(other_data_2, columns = columns, index = rows[5:9])
df4 = pd.DataFrame(other_data_3, columns = columns, index = ["Review 10", "Review 11", "Review 1","Review 2"])

### To display parts of the whole dataset for presentation

In [2]:
df

,length,helpful,emotional,Sentiment
Review 1,long,yes,yes,positive
Review 2,short,no,no,negative
Review 3,long,yes,yes,positive
Review 4,long,yes,no,negative
Review 5,long,no,yes,positive
Review 6,short,yes,yes,positive
Review 7,long,no,no,negative
Review 8,long,no,no,positive
Review 9,long,yes,yes,positive
Review 10,long,no,no,positive


### Helper functions 

In [3]:
### helper function to check how many labels my ds has and if a value is numeric

def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

class_counts(df)

def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

### Class for question to partitiate a dataset

In [4]:
## class is created
"""A Question is used to partition a dataset.
This class just records a 'column number' (e.g., 0 for Color) and a
'column value' (e.g., Green). The 'match' method is used to compare
the feature value in an example to the feature value stored in the
question. See the demo below.
"""
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        return "Is %s %s %s?" % (
            columns[self.column], condition, str(self.value))
    

In [5]:
## these questions are splitting nodes (e.g. split on attribute length(index = 0))
Question(0, "long")

Is length == long?

## Lets partitiate our dataset according to a randomly picked question

In [7]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows



def transform_data_to_df(data):
    df = pd.DataFrame(data, columns = columns)
    return df

In [10]:
#######
# Demo:
# Let's partition the training data based on whether item of dataset has length short.
true_rows, false_rows = partition(data, Question(2, 'yes'))
# true_rows will contain all the short reviews.

### e.g. split dataset on question emotional (index == 2) and display splitted datasets
true_rows_df = transform_data_to_df(true_rows)
true_rows_df

,length,helpful,emotional,Sentiment
0,long,yes,yes,positive
1,long,yes,yes,positive
2,long,no,yes,positive
3,short,yes,yes,positive
4,long,yes,yes,positive


In [11]:
false_rows_df = transform_data_to_df(false_rows)
false_rows_df

,length,helpful,emotional,Sentiment
0,short,no,no,negative
1,long,yes,no,negative
2,long,no,no,negative
3,long,no,no,positive
4,long,no,no,positive
5,long,no,no,positive


In [12]:
### another example --> split only  emotional == no rows on attribute helpful in next step
true_rows, false_rows = partition(false_rows, Question(1, 'yes'))

In [13]:
true_rows_df = transform_data_to_df(true_rows)
true_rows_df

,length,helpful,emotional,Sentiment
0,long,yes,no,negative


In [14]:
false_rows_df = transform_data_to_df(false_rows)
false_rows_df

,length,helpful,emotional,Sentiment
0,short,no,no,negative
1,long,no,no,negative
2,long,no,no,positive
3,long,no,no,positive
4,long,no,no,positive


In [15]:
## last example --> split only emotional == no && helpful == no rows on length attribute
true_rows, false_rows = partition(false_rows, Question(0, 'short'))

In [17]:
true_rows_df = transform_data_to_df(true_rows)
true_rows_df

,length,helpful,emotional,Sentiment
0,short,no,no,negative


In [16]:
false_rows = transform_data_to_df(false_rows)
false_rows

,length,helpful,emotional,Sentiment
0,long,no,no,negative
1,long,no,no,positive
2,long,no,no,positive
3,long,no,no,positive


# Different approaches to find best possible splitting attribute (either do gini impurity or Entropy)

In [18]:
### return the gini impurity of a row --> we do this to get the best possible splitting attribute

def gini(rows):
    """
    Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. 
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity
    


In [19]:
import math

In [20]:
def entropy(rows):
    
    """Calculate the Information Gain of the split with Entropy

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    slides .. in the beginning of the presentation
    """
    
    counts = class_counts(rows)
    info_gain = 0
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        info_gain += (-prob_of_lbl)  * math.log2(prob_of_lbl)   
    return info_gain

In [21]:

current_uncertainty = gini(data)
current_uncertainty


0.39669421487603307

In [22]:
second_uncertainty = entropy(data)
second_uncertainty

0.8453509366224364

In [25]:

def info_gain_gini(left, right, current_uncertainty):
    
    """
    Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)
    

In [26]:
def info_gain_entropy(left, right, current_info_gain):
    p = float(len(left)) / (len(left) + len(right))
    
    return current_info_gain - p * entropy(left) - (1 - p) * entropy(right)

In [28]:
# How much information do we gain by partioning on length?
true_rows, false_rows = partition(data, Question(0, 'long'))
info_gain_entropy(true_rows, false_rows, second_uncertainty)

0.03827452220629257

### Find best splitting attribute for a dataset by calculating entropy

In [29]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = entropy(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain_entropy(true_rows, false_rows, current_uncertainty)
            print("run")
            
            print(question)
            print(gain)
            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain > best_gain:
                best_gain, best_question = gain, question
                print("update")
               

    return best_gain, best_question

### Test run to find the best splitting attr. for the first partition

In [30]:
# Find the best question to ask first for our toy dataset.
best_gain, question = find_best_split(data)
print(question)
print(best_gain)

run
Is length == long?
0.03827452220629257
update
run
Is length == short?
0.03827452220629268
update
run
Is helpful == no?
0.016313165825732
run
Is helpful == yes?
0.016313165825732057
run
Is emotional == no?
0.299896391167891
update
run
Is emotional == yes?
0.299896391167891
Is emotional == no?
0.299896391167891


## Test run no.2 to find best splitting attr. for another dataset -- just to check if the entropy calculation works

In [31]:
data2 =[ 
    ["long", "yes", "no", "negative"],
    ["long", "no", "no", "negative"],
    ["long", "no", "no", "positive"], 
    ["long", "no", "no", "positive"],
    ["long", "no", "no", "positive"],
]

In [32]:
best_gain, question = find_best_split(data2)
print(question)
print(best_gain)

run
Is helpful == no?
0.3219280948873623
update
run
Is helpful == yes?
0.3219280948873623
Is helpful == no?
0.3219280948873623


## Create leafs of our decision tree and also nodes with splitting questions

In [35]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [36]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

## Functions to build a decision tree and show which splits should be done to get best results

In [37]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)
    print(gain)
    print(question)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [38]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [39]:
my_tree = build_tree(data)
my_tree

run
Is length == long?
0.03827452220629257
update
run
Is length == short?
0.03827452220629268
update
run
Is helpful == no?
0.016313165825732
run
Is helpful == yes?
0.016313165825732057
run
Is emotional == no?
0.299896391167891
update
run
Is emotional == yes?
0.299896391167891
0.299896391167891
Is emotional == no?
run
Is length == long?
0.19087450462110944
update
run
Is length == short?
0.19087450462110944
run
Is helpful == no?
0.19087450462110944
run
Is helpful == yes?
0.19087450462110944
0.19087450462110944
Is length == long?
run
Is helpful == no?
0.3219280948873623
update
run
Is helpful == yes?
0.3219280948873623
0.3219280948873623
Is helpful == no?
0
None
0
None
0
None
run
Is length == long?
0.0
run
Is length == short?
0.0
run
Is helpful == no?
0.0
run
Is helpful == yes?
0.0
0
None


In [40]:
print_tree(my_tree)

Is emotional == no?
--> True:
  Is length == long?
  --> True:
    Is helpful == no?
    --> True:
      Predict {'negative': 1, 'positive': 3}
    --> False:
      Predict {'negative': 1}
  --> False:
    Predict {'negative': 1}
--> False:
  Predict {'positive': 5}


## Make classification of new tuples according to our before created dec tree

In [41]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [42]:
classify(["long", "no", "yes"], my_tree)

{'positive': 5}